In [50]:

import numpy as np
import pandas as pd
!pip install tensorflow
import tensorflow as tf
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import tensorflow as tf
!pip install keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [51]:
dataset = pd.read_csv("/content/drive/MyDrive/eng_-french.csv")
dataset.head()
dataset = dataset.drop(dataset.index[10000:175621])



In [52]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   English words/sentences  10000 non-null  object
 1   French words/sentences   10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [53]:
sentences_en = dataset["English words/sentences"].to_numpy()
sentences_fr = dataset["French words/sentences"].to_numpy()

valid_fraction = 0.1
valid_len = int(valid_fraction * len(dataset))

sentences_en_train = sentences_en[:-valid_len]
sentences_fr_train = sentences_fr[:-valid_len]

sentences_en_valid = sentences_en[-valid_len:]
sentences_fr_valid = sentences_fr[-valid_len:]
print(sentences_en)
print(sentences_fr)

['Hi.' 'Run!' 'Run!' ... 'Be very careful.' 'Be very careful.'
 'Be very careful.']
['Salut!' 'Cours\u202f!' 'Courez\u202f!' ... 'Soyez très prudente !'
 'Soyez très prudents !' 'Soyez très prudentes !']


In [67]:
def prepare_input_and_target(sentences_en, sentences_fr):

    return (sentences_en, b"startofseq " + sentences_fr), sentences_fr + b" endofseq"


def from_sentences_dataset(
    sentences_en,
    sentences_fr,
    batch_size=32,
    cache=True,
    shuffle=False,
    shuffle_buffer_size=10_000,
    seed=None,
):

    dataset = tf.data.Dataset.from_tensor_slices((sentences_en, sentences_fr))
    dataset = dataset.map(prepare_input_and_target, num_parallel_calls=tf.data.AUTOTUNE)
    if cache:
        dataset = dataset.cache()
    if shuffle:
        dataset = dataset.shuffle(shuffle_buffer_size, seed=seed)
    return dataset.batch(batch_size)

In [55]:
benchmark_ds = from_sentences_dataset(sentences_en_train, sentences_fr_train)
benchmark_ds = benchmark_ds.prefetch(tf.data.AUTOTUNE)
bench_results = tfds.benchmark(benchmark_ds, batch_size=32)


************ Summary ************



  0%|          | 0/282 [00:00<?, ?it/s]

Examples/sec (First included) 10044.24 ex/sec (total: 9056 ex, 0.90 sec)
Examples/sec (First only) 431.03 ex/sec (total: 32 ex, 0.07 sec)
Examples/sec (First excluded) 10906.84 ex/sec (total: 9024 ex, 0.83 sec)


In [56]:
input_texts = dataset["English words/sentences"].tolist()
target_texts = dataset["French words/sentences"].tolist()

In [57]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess_data(input_texts, target_texts):
    input_tokenizer = Tokenizer()
    input_tokenizer.fit_on_texts(input_texts)
    input_sequences = input_tokenizer.texts_to_sequences(input_texts)
    input_sequences = pad_sequences(input_sequences, padding='post')

    target_tokenizer = Tokenizer()
    target_tokenizer.fit_on_texts(target_texts)
    target_sequences = target_tokenizer.texts_to_sequences(target_texts)
    target_sequences = pad_sequences(target_sequences, padding='post')

    return input_sequences, target_sequences, input_tokenizer, target_tokenizer




In [58]:
input_sequences, target_sequences, input_tokenizer, target_tokenizer = preprocess_data(input_texts, target_texts)

In [59]:
def build_model(input_vocab_size, target_vocab_size, units):
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_vocab_size, units)(encoder_inputs)
    encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(target_vocab_size, units)(decoder_inputs)
    decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

    attention = Attention()
    context_vector = attention([decoder_outputs, encoder_outputs])
    decoder_combined_context = tf.concat([context_vector, decoder_outputs], axis=-1)

    decoder_dense = Dense(target_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_combined_context)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

input_vocab_size = 10000
target_vocab_size = 10000


In [60]:
vocab_size = 10000  # Replace with the actual vocabulary size
max_length = 100  # Replace with the actual maximum length of input sequences
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

In [61]:
model_1 = build_model(input_vocab_size=len(input_tokenizer.word_index) + 1,
                    target_vocab_size=len(target_tokenizer.word_index) + 1,
                    units=204)

In [62]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
print(f"Input sequences shape: {input_sequences.shape}")
print(f"Target sequences shape: {target_sequences.shape}")
model = Sequential([
  layers.Embedding(vocab_size, 128, input_length=max_seq_len-1),
  layers.LSTM(128, return_sequences=True),
  layers.LSTM(128),
  layers.Dense(vocab_size, activation='softmax')])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

model.fit([input_sequences, target_sequences], target_sequences, batch_size=64, epochs=5, validation_split=0.5)

Input sequences shape: (10000, 5)
Target sequences shape: (10000, 10)


NameError: name 'layers' is not defined

In [65]:
from nltk.translate.bleu_score import corpus_bleu

translations = ["Sault"]
reference_translations = ["hello!"]

# Compute BLEU score
bleu_score = corpus_bleu(reference_translations, translations)

# Print BLEU score
print("BLEU Score:", bleu_score)


BLEU Score: 1.2183324802375697e-231


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning:


The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning:


The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning:


The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()

